In [1]:
import librosa
import librosa.display
import numpy as np
import pandas as pd
import os
from typing import List, Tuple, Dict
import matplotlib.pyplot as plt
import random
import cv2
# from vgg16 import VGG16_LSTM
import pickle
import torch.nn as nn
import torch
import datetime
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms, utils, models
from torch.utils.data import Dataset, DataLoader
import logging
import torch
import torch.nn as nn
from qx_noface_no2d_video_swin_transformer01 import SwinTransformer3D
import gc
from einops import rearrange
from tqdm import tqdm
from tqdm.contrib.logging import logging_redirect_tqdm
batchsz = 16
frame_nums=15
start_epoch = 0
test_flag=False
model_name ="AST_VST——text batchsize4"
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('[%(asctime)s]::%(module)s::%(levelname)s::%(message)s')
streamHandler = logging.StreamHandler()
streamHandler.setFormatter(formatter)
fileHandler = logging.FileHandler('./Logs/personalityLog.log')
fileHandler.setFormatter(formatter)
logger.addHandler(streamHandler)
logger.addHandler(fileHandler)
USE_CUDA = torch.cuda.is_available()
os.environ["TOKENIZERS_PARALLELISM"] = "false"
torch.backends.cudnn.enabled = False

In [2]:
logger.debug('============={}+Video swin transformerTrain Strat!============='.format(model_name))
USE_CUDA = torch.cuda.is_available()
print(USE_CUDA)

device = torch.device('cuda' if USE_CUDA else 'cpu')
print('학습을 진행하는 기기:', device)

[2025-07-05 16:02:37,860]::1779168656::DEBUG::=============AST_VST——text batchsize4+Video swin transformerTrain Strat!=============


True
학습을 진행하는 기기: cuda


In [3]:
def getdata(pathname,datatype):
    if datatype=="train":
        final_data_set=[]
        output_file=open(pathname, "rb")
        for i in range(1):
            # train_data_set.extend(pickle.load(output_file))
            final_data_set.extend(pickle.load(output_file))
            # print(i)
        return final_data_set
    elif datatype=="valid":
        final_data_set=[]
        output_file=open(pathname, "rb")
        for i in range(1):
            valid_data_set=[]
            final_data_set.extend(pickle.load(output_file))
            del valid_data_set
        return final_data_set
print(1)

1


In [4]:
def datasize(type,batchsz):
    if type=='train':
        i=5076#5967
        while True:
            if i%batchsz==0:
                return i
            else:
                i-=1
    else:
        i=1700#1986
        while True:
            if i%batchsz==0:
                return i
            else:
                i-=1
print(1)

1


In [5]:
save_model_file_path = '/home/user/10TB/personalityAI/multimodal/KETI2025_finetuned_videoswintransformer/{}_{}.{}'
#mtcnn= MTCNN(image_size=224, margin=0, min_face_size= 20, 
#             thresholds=[0.6, 0.7, 0.7], post_process=True)   # 안씀
print(1)

1


#### dataset 구성

0 : audio   
1 : fullshot video   
2 : face   
3 : lip   
4 : text   
5 : tag   

In [6]:
def reshape_to_expected_traininput(dataset: List[Tuple[ np.ndarray, np.ndarray]]) -> Tuple[
     np.ndarray, np.ndarray]:
    result=[]
    for i in range(0, len(dataset[0])):
        result.append((dataset[0][i],dataset[1][i],dataset[2][i],dataset[3][i]))
    result = result[:datasize("train",batchsz)]
    x0_list = []
    x1_list = []
    x2_list = []
    x3_list = []

    for i in range(0, len(result)):
        x0_list.append(result[i][0])
        x1_list.append(result[i][1])
        x2_list.append(result[i][2])
        x3_list.append(result[i][3])

    return (np.stack(x0_list), np.stack(x1_list), np.stack(x2_list),np.stack(x3_list))
print(1)

1


In [7]:
def reshape_to_expected_validinput(dataset: List[Tuple[ np.ndarray, np.ndarray]]) -> Tuple[
     np.ndarray, np.ndarray]:
    result=[]
    for i in range(0, len(dataset[0])):
        result.append((dataset[0][i],dataset[1][i],dataset[2][i],dataset[3][i]))
    result = result[:datasize("valid",batchsz)]
    x0_list = []
    x1_list = []
    x2_list = []
    x3_list = []

    for i in range(0, len(result)):
        x0_list.append(result[i][0])
        x1_list.append(result[i][1])
        x2_list.append(result[i][2])
        x3_list.append(result[i][3])

    return (np.stack(x0_list), np.stack(x1_list), np.stack(x2_list),np.stack(x3_list))
print(1)

1


In [8]:
import pickle


# load
with open('Your Path', 'rb') as f4:
    train_set_data = pickle.load(f4)

# load
with open('Your Path', 'rb') as f5:
    validation_set_data = pickle.load(f5)

# load
with open('Your Path', 'rb') as f6:
    test_set_data = pickle.load(f6)

In [9]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
])
print(1)

1


In [10]:
# 데이터 텐서로 만들기

class ChalearnDataset(Dataset):  
    def __init__(self,audiodata,fullshot,textdata,tagdata,transform=None):
        self.audiodata=audiodata
        self.fullshot=fullshot
        self.tagdata=tagdata
        self.textdata=textdata
        self.transform = transform  # 표준화 여부
    def __len__(self):
        return len(self.fullshot)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()#텐서의 경우 목록으로 돌아가기
        audiodata = self.audiodata[idx]
        audiodata = torch.FloatTensor(audiodata)
        fullshot=self.fullshot[idx]
        fullshot=torch.FloatTensor(fullshot)
        textdata = self.textdata[idx]
        # image_data=image_data.reshape(15,3,224,224)
        big_five_sorces=self.tagdata[idx]
        big_five_sorces = torch.FloatTensor((big_five_sorces-24)/(120-24))
        return audiodata,fullshot,textdata,big_five_sorces
print(1)

1


In [11]:
num_workerssz = 12
lr = 1e-04
epochs = 15  #  120
print(1)

1


In [12]:
train_set_data = ChalearnDataset(audiodata=train_set_data[0],
                                 fullshot = train_set_data[1],
                                 textdata=train_set_data[2],
                                 tagdata=train_set_data[3],
                                 transform=transform)
val_set_data = ChalearnDataset(audiodata=validation_set_data[0],
                               fullshot = validation_set_data[1],
                               textdata=validation_set_data[2],
                               tagdata=validation_set_data[3],
                               transform=transform)
test_set_data = ChalearnDataset(audiodata=test_set_data[0],
                               fullshot = test_set_data[1],
                               textdata=test_set_data[2],
                               tagdata=test_set_data[3],
                               transform=transform)

train_dataloader = DataLoader(dataset=train_set_data, batch_size=batchsz, 
                              shuffle=True, num_workers=num_workerssz)
val_dataloader = DataLoader(dataset=val_set_data, batch_size=batchsz, 
                            shuffle=True, num_workers=num_workerssz)
test_dataloader = DataLoader(dataset=test_set_data, batch_size=batchsz, 
                            shuffle=True, num_workers=num_workerssz)
max_value=0
print(1)

1


In [13]:
import torch.nn.parallel

model=SwinTransformer3D()
model = nn.DataParallel(model)

model.to(device)
criterion = torch.nn.L1Loss().to(device)  # 손실함수
optimizer = torch.optim.AdamW(model.parameters(), lr=lr) # 옵티마이저

if test_flag==True:
    checkpoint=torch.load(save_model_file_path.format('model',start_epoch,'pth'), map_location=device)
    model.load_state_dict(checkpoint["model"])
    optimizer.load_state_dict(checkpoint["optimizer"])
    # criterion.load_state_dict(checkpoint["loss"])
train_avg_loss0=[]
val_avg_loss0=[]
print(1)

Some weights of RobertaModel were not initialized from the model checkpoint at /home/egg2018037024/Jupyter_Home/Multi-modal_personality/koRoBERTa_base-checkpoint16000 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


---------------AST Model Summary---------------
ImageNet pretraining: True, AudioSet pretraining: True
audioset_pretrain == True
[PE] Not use imagenet pretrain
frequncey stride=10, time stride=10
number of patches=599
Text model params :  111208704
Audio model params :  88731728
Text+Audio prams:  199940432


/home/egg2018037024/venv310/lib/python3.10/site-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


1


In [14]:
from torchinfo import summary

summary(model)

Layer (type:depth-idx)                                                      Param #
DataParallel                                                                --
├─SwinTransformer3D: 1-1                                                    --
│    └─TextBlock: 2-1                                                       --
│    │    └─RobertaModel: 3-1                                               110,618,112
│    │    └─Linear: 3-2                                                     590,592
│    │    └─Dropout: 3-3                                                    --
│    └─VideoResNet: 2-2                                                     --
│    │    └─R2Plus1dStem: 3-4                                               15,473
│    │    └─Sequential: 3-5                                                 444,032
│    │    └─Sequential: 3-6                                                 1,470,360
│    │    └─Sequential: 3-7                                                 5,874,736
│    │    

In [15]:
def extract_ocean(data, batch_size):
    o=[]
    c=[]
    e=[]
    a=[]
    n=[]
    
    for i in range(batch_size):
        try:
            o.append(data[i][0].item())
            c.append(data[i][1].item())
            e.append(data[i][2].item())
            a.append(data[i][3].item())
            n.append(data[i][4].item())
            
        except Exception as err:
            print(err)
            pass
        
    o = torch.tensor(o) 
    c = torch.tensor(c)
    e = torch.tensor(e) 
    a = torch.tensor(a) 
    n = torch.tensor(n) 
    return o,c,e,a,n

In [16]:
results = []
with torch.cuda.device(0):
    for i in range(start_epoch, epochs):
        train_avg_loss = 0
        val_avg_loss = 0
        train_avg_o_loss = 0
        train_avg_c_loss = 0
        train_avg_e_loss = 0
        train_avg_a_loss = 0
        train_avg_n_loss = 0
        val_avg_o_loss = 0
        val_avg_c_loss = 0
        val_avg_e_loss = 0
        val_avg_a_loss = 0
        val_avg_n_loss = 0
        
        for audiodata, fullshot, textdata, big_five_data in tqdm(train_dataloader):
            big_five_data=big_five_data.permute(0,2,1).contiguous()
            big_five_data=big_five_data.squeeze(1)
            audiodata = audiodata.to(device)
            fullshot = fullshot.to(device)
            big_five_data = big_five_data.to(device)
            optimizer.zero_grad()  # 기울기가 0이 됩니다.
            hypothesis = model(fullshot,textdata,audiodata)  # 모델의 예측 결과를 저장합니다.
            loss = criterion(hypothesis, big_five_data)  # 예측된 결과와 실제 태그 사이의 손실 값을 저장합니다.
            loss.backward()  # 역방향 전파입니다. 
            optimizer.step()  
            train_avg_loss += loss.item()
        train_avg_loss=train_avg_loss/len(train_dataloader)
        
        with torch.no_grad(): #validate
            for audiodata,fullshot,textdata,big_five_data in tqdm(val_dataloader):
                big_five_data=big_five_data.permute(0,2,1).contiguous()
                big_five_data=big_five_data.squeeze(1)
                audiodata = audiodata.to(device)
                fullshot = fullshot.to(device)
                big_five_data = big_five_data.to(device)
                hypothesis = model(fullshot,textdata,audiodata)
                val_loss = criterion(hypothesis, big_five_data)
                hypothesiso,hypothesisc,hypothesise,hypothesisa,hypothesisn=extract_ocean(hypothesis,batchsz)
                big_five_datao,big_five_datac,big_five_datae,big_five_dataa,big_five_datan=extract_ocean(big_five_data,batchsz)
                val_o_loss = criterion(hypothesiso, big_five_datao) 
                val_c_loss = criterion(hypothesisc, big_five_datac) 
                val_e_loss = criterion(hypothesise, big_five_datae) 
                val_a_loss = criterion(hypothesisa, big_five_dataa) 
                val_n_loss = criterion(hypothesisn, big_five_datan)  
                val_avg_loss += val_loss.item()
                val_avg_o_loss += val_o_loss.item()
                val_avg_c_loss += val_c_loss.item()
                val_avg_e_loss += val_e_loss.item()
                val_avg_a_loss += val_a_loss.item()
                val_avg_n_loss += val_n_loss.item()
            val_avg_loss=val_avg_loss/len(val_dataloader)
            val_avg_o_loss=val_avg_o_loss/len(val_dataloader)
            val_avg_c_loss=val_avg_c_loss/len(val_dataloader)
            val_avg_e_loss=val_avg_e_loss/len(val_dataloader)
            val_avg_a_loss=val_avg_a_loss/len(val_dataloader)
            val_avg_n_loss=val_avg_n_loss/len(val_dataloader)
        torch.cuda.empty_cache()
        # if (1-val_avg_loss)>=(1-max_value):
        max_value=val_avg_loss
        start_epoch+=1
 #       if (i + 1) % 5 == 0:
        torch.save({
                'epoch': i+1,
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'loss': val_avg_loss,
            }, save_model_file_path.format('model',start_epoch,'pth'))
        results.append([i+1, train_avg_loss, val_avg_loss, 1-val_avg_loss, 1-val_avg_o_loss, 1-val_avg_c_loss, 1-val_avg_e_loss, 1-val_avg_a_loss,1-val_avg_n_loss])
        logger.debug('\n Epoch: {}, \n train_loss: {:.4f}, \n val_loss: {:.4f}, \n 1-MAE: {:.4f} , \n 1-val_avg_o_loss:{:.4f} , \n 1-val_avg_c_loss:{:.4f} , \n 1-val_avg_e_loss:{:.4f} , \n 1-val_avg_a_loss:{:.4f} , \n 1-val_avg_n_loss:{:.4f} \n'.format(i+1, train_avg_loss, val_avg_loss, 1-val_avg_loss, 1-val_avg_o_loss, 1-val_avg_c_loss, 1-val_avg_e_loss, 1-val_avg_a_loss,1-val_avg_n_loss))
    

100%|██████████| 318/318 [02:47<00:00,  1.90it/s]
[2025-07-05 16:22:38,698]::1823589130::DEBUG::
 Epoch: 1, 
 train_loss: 0.1384, 
 val_loss: 0.1262, 
 1-MAE: 0.8738 , 
 1-val_avg_o_loss:0.8699 , 
 1-val_avg_c_loss:0.8865 , 
 1-val_avg_e_loss:0.8330 , 
 1-val_avg_a_loss:0.8902 , 
 1-val_avg_n_loss:0.8896 

100%|██████████| 318/318 [02:47<00:00,  1.90it/s]
[2025-07-05 16:42:02,845]::1823589130::DEBUG::
 Epoch: 2, 
 train_loss: 0.1145, 
 val_loss: 0.1154, 
 1-MAE: 0.8846 , 
 1-val_avg_o_loss:0.8729 , 
 1-val_avg_c_loss:0.8823 , 
 1-val_avg_e_loss:0.8682 , 
 1-val_avg_a_loss:0.9078 , 
 1-val_avg_n_loss:0.8917 

100%|██████████| 318/318 [02:47<00:00,  1.90it/s]
[2025-07-05 17:01:27,650]::1823589130::DEBUG::
 Epoch: 3, 
 train_loss: 0.1109, 
 val_loss: 0.1199, 
 1-MAE: 0.8801 , 
 1-val_avg_o_loss:0.8682 , 
 1-val_avg_c_loss:0.8867 , 
 1-val_avg_e_loss:0.8455 , 
 1-val_avg_a_loss:0.9117 , 
 1-val_avg_n_loss:0.8883 

100%|██████████| 318/318 [02:47<00:00,  1.90it/s]
[2025-07-05 17:20:51,764]:

### Test

In [18]:
test_model = SwinTransformer3D()
test_model = nn.DataParallel(test_model)
state_dict_m = torch.load("/home/user/10TB/personalityAI/multimodal/KETI2025_finetuned_videoswintransformer/model_14.pth")  

with torch.cuda.device(0):
    test_model.load_state_dict(state_dict_m['model'])
    test_model.to(device)
    test_model.eval()
    
    test_avg_loss = 0
    test_avg_o_loss = 0
    test_avg_c_loss = 0
    test_avg_e_loss = 0
    test_avg_a_loss = 0
    test_avg_n_loss = 0
    
    with torch.no_grad():#validate
        for audiodata,fullshot,textdata,big_five_data in tqdm(test_dataloader):
            big_five_data=big_five_data.permute(0,2,1).contiguous()
            big_five_data=big_five_data.squeeze()
            audiodata = audiodata.to(device)
            fullshot = fullshot.to(device)
            big_five_data = big_five_data.to(device)
            hypothesis = test_model(fullshot,textdata,audiodata)
            test_loss = criterion(hypothesis, big_five_data)
            hypothesiso,hypothesisc,hypothesise,hypothesisa,hypothesisn=extract_ocean(hypothesis,batchsz)
            big_five_datao,big_five_datac,big_five_datae,big_five_dataa,big_five_datan=extract_ocean(big_five_data,batchsz)
            test_o_loss = criterion(hypothesiso, big_five_datao) 
            test_c_loss = criterion(hypothesisc, big_five_datac) 
            test_e_loss = criterion(hypothesise, big_five_datae) 
            test_a_loss = criterion(hypothesisa, big_five_dataa) 
            test_n_loss = criterion(hypothesisn, big_five_datan)  
            
            test_avg_loss += test_loss
            test_avg_o_loss += test_o_loss
            test_avg_c_loss += test_c_loss
            test_avg_e_loss += test_e_loss
            test_avg_a_loss += test_a_loss
            test_avg_n_loss += test_n_loss
            
        test_avg_loss = test_avg_loss / len(test_dataloader)
        test_avg_o_loss = test_avg_o_loss / len(test_dataloader)
        test_avg_c_loss = test_avg_c_loss / len(test_dataloader)
        test_avg_e_loss = test_avg_e_loss / len(test_dataloader)
        test_avg_a_loss = test_avg_a_loss / len(test_dataloader)
        test_avg_n_loss = test_avg_n_loss / len(test_dataloader)
        
    torch.cuda.empty_cache()
    # if (1-val_avg_loss)>=(1-max_value):
    print('1-MAE: {:.4f} , 1-o_loss:{:.4f} , 1-c_loss:{:.4f} , 1-e_loss:{:.4f} , 1-a_loss:{:.4f} , 1-n_loss:{:.4f}'.format(1-test_avg_loss, 1-test_avg_o_loss, 1-test_avg_c_loss, 1-test_avg_e_loss, 1-test_avg_a_loss,1-test_avg_n_loss))

Some weights of RobertaModel were not initialized from the model checkpoint at /home/egg2018037024/Jupyter_Home/Multi-modal_personality/koRoBERTa_base-checkpoint16000 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


---------------AST Model Summary---------------
ImageNet pretraining: True, AudioSet pretraining: True
audioset_pretrain == True
[PE] Not use imagenet pretrain
frequncey stride=10, time stride=10
number of patches=599
Text model params :  111208704
Audio model params :  88731728
Text+Audio prams:  199940432


100%|██████████| 318/318 [02:47<00:00,  1.90it/s]


1-MAE: 0.8839 , 1-o_loss:0.8908 , 1-c_loss:0.8866 , 1-e_loss:0.8605 , 1-a_loss:0.9083 , 1-n_loss:0.8731
